# Data Preprocessing for Restaurant Reviews Analysis

## Overview
This notebook performs data preprocessing and integration of restaurant reviews data collected from multiple sources. The main objective is to clean, standardize, and combine review data with restaurant details to create a comprehensive dataset for analysis.

## Process Flow
1. **Data Import and Initial Processing**

2. **Data Standardization**
   - Standardizes column names across different data formats
   - Merges the two groups of data after standardization

3. **Data Cleaning**
   - Removes duplicate reviews
   - Eliminates entries without review content or dates
   - Processes and standardizes restaurant names
   - Converts review dates to a consistent datetime format using reference dates

4. **Data Integration**
   - Merges review data with restaurant details (categories, addresses, etc.)
   - Handles missing values and special cases
   - Creates a final consolidated dataset

## Final Dataset Statistics
- Total number of processed reviews: 171,042
- Key columns:
  - Mekan_Adı (Restaurant Name)
  - Kullanıcı_Adı (Username)
  - Yorum (Review)
  - Puan (Rating)
  - Yorum_Tarihi (Review Date)
  - Restoran_Kategorisi (Restaurant Category)
  - Restoran_Toplam_Yorum_Sayısı (Total Restaurant Reviews)
  - Ortalama_Restoran_Puanı (Average Restaurant Rating)
  - Restoran_Adresi (Restaurant Address)

## Output
The final preprocessed dataset is exported to "FinalDataFrame.xlsx" for further analysis.

In [ ]:
import pandas as pd
import os
import datetime
from collections import defaultdict
from dateutil.relativedelta import relativedelta
import re
import unicodedata
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
# csv_folder değişkeninin tanımlanması
csv_folder = "yorumlar"

# csv_files değişkeninin tanımlanması
csv_files = [os.path.join(csv_folder, f) for f in os.listdir(csv_folder) if f.endswith('.csv')]

# Tüm csv dosyalarına "csv_creation_date" sütununun eklenmesi
for file in csv_files:
    file_path = os.path.join(csv_folder, os.path.basename(file))
    try:
        # Dosyanın oluşturulma tarihi
        creation_time = os.path.getctime(file_path)
        creation_date = datetime.datetime.fromtimestamp(creation_time).strftime('%Y-%m-%d')

        df = pd.read_csv(file_path, sep=';', encoding='utf-8', on_bad_lines='skip')
        # Sütunu ekle
        df['csv_creation_date'] = creation_date
        # Dosyayı tekrar kaydet
        df.to_csv(file_path, sep=';', encoding='utf-8', index=False)
    except Exception as e:
        print(f"Hata oluştu (creation_date ekleme): {file_path} - {e}")

# Tüm CSV dosyalarına "Mekan_Adı" sütununun eklenmesi
for file in csv_files:
    file_path = os.path.join(csv_folder, os.path.basename(file))
    try:
        mekan_adi = os.path.splitext(os.path.basename(file))[0].replace("yorumlar_", "")
        df = pd.read_csv(file_path, sep=';', encoding='utf-8', on_bad_lines='skip')
        df['Mekan_Adı_Yeni'] = mekan_adi
        df.to_csv(file_path, sep=';', encoding='utf-8', index=False)
    except Exception as e:
        print(f"Hata oluştu (Mekan_Adı ekleme): {file_path} - {e}")

# Her CSV dosyasının sütunlarını kontrol ederek, aynı sütunlara sahip olanların gruplandırılması
columns_groups = defaultdict(list)

for file in csv_files:
    file_path = os.path.join(csv_folder, os.path.basename(file))
    try:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8', nrows=1, on_bad_lines='skip')
        columns_tuple = tuple(df.columns)
        columns_groups[columns_tuple].append(file)
    except Exception as e:
        print(f"Hata oluştu: {file_path} - {e}")

# Grupların ve her grubun sütun isimlerinin yazdırılması
for idx, (cols, files) in enumerate(columns_groups.items(), 1):
    print(f"Grup {idx}: {len(files)} dosya")
    print("Sütunlar:", cols)
    print("Örnek dosya:", files[0])

Grup 1: 1589 dosya
Sütunlar: ('Yorum_Sırası', 'Mekan_Adı', 'Mekan_Türü', 'Veri_Çekme_Tarihi', 'Kullanıcı_Adı', 'Puan', 'Yorum', 'Yorum_Tarihi', 'csv_creation_date', 'Mekan_Adı_Yeni')
Örnek dosya: yorumlar\yorumlar_01_adanali_ismail_ustanin_yeri.csv
Grup 2: 1687 dosya
Sütunlar: ('Yorum_Sırası', 'Kullanıcı Adı', 'Puan', 'Yorum', 'Yorum Tarihi', 'csv_creation_date', 'Mekan_Adı_Yeni')
Örnek dosya: yorumlar\yorumlar_100_more_alsancak.csv


In [ ]:
# columns_groups sözlüğündeki grupları iki ayrı dataframe olarak birleştirilmesi
group_keys = list(columns_groups.keys())

# Grup 1 ve Grup 2 dosyalarını alınması
group1_files = columns_groups[group_keys[0]]
group2_files = columns_groups[group_keys[1]]

# Her grup için dataframe'lerin okunması ve birleştirmesi
group1_dfs = [pd.read_csv(f, sep=';', encoding='utf-8', on_bad_lines='skip') for f in group1_files]
group2_dfs = [pd.read_csv(f, sep=';', encoding='utf-8', on_bad_lines='skip') for f in group2_files]

group1_df = pd.concat(group1_dfs, ignore_index=True)
group2_df = pd.concat(group2_dfs, ignore_index=True)

In [ ]:
group1_df.head(3)

,Yorum_Sırası,Mekan_Adı,Mekan_Türü,Veri_Çekme_Tarihi,Kullanıcı_Adı,Puan,Yorum,Yorum_Tarihi,csv_creation_date,Mekan_Adı_Yeni
0,1,01 Adanalı İsmail Usta'nın Yeri,Restoran,15.07.2025 15:56,güvenç tünay,5.0,Mezeler etler tavuk harika\nHizmet süper\nHarika,bir hafta önce,2025-07-20,01_adanali_ismail_ustanin_yeri
1,2,01 Adanalı İsmail Usta'nın Yeri,Restoran,15.07.2025 15:56,Ali Can Vurgun,5.0,Mezeleri temizliği yemeklerin lezzeti harikadi...,bir ay önce,2025-07-20,01_adanali_ismail_ustanin_yeri
2,3,01 Adanalı İsmail Usta'nın Yeri,Restoran,15.07.2025 15:56,Pelin Yildirim,1.0,Resmen dolandırıldık. Yani bunu başka türlü aç...,bir ay önce,2025-07-20,01_adanali_ismail_ustanin_yeri


In [ ]:
group2_df.head(3)

,Yorum_Sırası,Kullanıcı Adı,Puan,Yorum,Yorum Tarihi,csv_creation_date,Mekan_Adı_Yeni
0,1,Songül,5.0,Sushileri gerçekten cok basarili izmir roll fa...,10 saat önce,2025-07-20,100_more_alsancak
1,2,Elif Çoban,5.0,Rahat oturma düzeni kibar servis personeli tem...,10 saat önce,2025-07-20,100_more_alsancak
2,3,Sudenaz Arslan,5.0,keyifli bir deneyimdi. yemekleri güzel ve pers...,10 saat önce,2025-07-20,100_more_alsancak


In [ ]:
# Dataframe'leri birleştirmeden önce, birleştirilecek sütunların seçilmesi ve gruplar arasındaki sütun isimlerinin aynı hale getirilmesi

group1_df = group1_df[['Yorum_Sırası', 'Kullanıcı_Adı', 'Puan', 'Yorum', 'Yorum_Tarihi', 'csv_creation_date', 'Mekan_Adı_Yeni']]
group2_df.columns = ['Yorum_Sırası', 'Kullanıcı_Adı', 'Puan', 'Yorum', 'Yorum_Tarihi', 'csv_creation_date', 'Mekan_Adı_Yeni']

In [ ]:
# group1_df ve group2_df'nin alt alta birleştirilmesi
combined_df = pd.concat([group1_df, group2_df], ignore_index=True)

# Yinelenen satırları kaldırılması (Kullanıcı adı, puan, yorum ve yorum tarihi sütunları aynı olanara göre)
combined_df = combined_df.drop_duplicates(subset=["Kullanıcı_Adı", "Puan", "Yorum", "Yorum_Tarihi"], ignore_index=True)

# Yorum Sırası sütunu çalışmalarda kullanılmadığı için kaldırması
combined_df.drop(columns=['Yorum_Sırası'], inplace=True)

# Yorum bilgisi içermeyen satırların kaldırılması
combined_df = combined_df.dropna(subset=['Yorum'])

# Yorum tarihi içermeyen satırların kaldırması
combined_df = combined_df.dropna(subset=['Yorum_Tarihi'])

In [ ]:
# Temizlik işlemleri sonrası verinin kontrol edilmesi

combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177302 entries, 0 to 226248
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Kullanıcı_Adı      177301 non-null  object 
 1   Puan               177302 non-null  float64
 2   Yorum              177302 non-null  object 
 3   Yorum_Tarihi       177302 non-null  object 
 4   csv_creation_date  177302 non-null  object 
 5   Mekan_Adı_Yeni     177302 non-null  object 
dtypes: float64(1), object(5)
memory usage: 9.5+ MB


*serra: burada aynı restoran isimlileri ne yaptı incelemek lazım-serra*

## Yorum Tarihi sütununu "csv_creation_date" bilgisini kullanarak tarih formatına çevirme

In [ ]:
# Tarihleri datetime tipine çevir
combined_df["csv_creation_date"] = pd.to_datetime(combined_df["csv_creation_date"], dayfirst=True)

# Türkçe yazılı rakam sözlüğü
yazili_sayilar = {
    "bir": 1,
    "iki": 2,
    "üç": 3,
    "dört": 4,
    "beş": 5,
    "altı": 6,
    "yedi": 7,
    "sekiz": 8,
    "dokuz": 9,
    "on": 10
}

def sayi_coz(metin_parca):
    # Sayıya çevrilebiliyorsa direkt çevir
    try:
        return int(metin_parca)
    except:
        # Yazılı sayılardan bulmaya çalış
        return yazili_sayilar.get(metin_parca.lower(), 0)

def hesapla_gercek_tarih(metin, referans):
    parcala = metin.lower().split()
    miktar = sayi_coz(parcala[0])

    if "ay" in metin:
        return referans - relativedelta(months=miktar)
    elif "yıl" in metin:
        return referans - relativedelta(years=miktar)
    elif "hafta" in metin:
        return referans - pd.Timedelta(weeks=miktar)
    elif "gün" in metin:
        return referans - pd.Timedelta(days=miktar)
    elif "saat" in metin:
        return referans  # aynı gün
    else:
        return referans

combined_df["Yorum_Tarihi_Yeni"] = combined_df.apply(lambda x: hesapla_gercek_tarih(x["Yorum_Tarihi"], x["csv_creation_date"]), axis=1)

# Tarih formatını gün.ay.yıl olarak yaz
combined_df["Yorum_Tarihi_Yeni"] = combined_df["Yorum_Tarihi_Yeni"].dt.strftime("%d.%m.%Y")
combined_df["Yorum_Tarihi_Yeni"] = pd.to_datetime(combined_df["Yorum_Tarihi_Yeni"], dayfirst=True)

combined_df.drop(columns=['Yorum_Tarihi', 'csv_creation_date'], inplace=True)

C:\Users\Burak\AppData\Local\Temp\ipykernel_1652\348111762.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  combined_df["csv_creation_date"] = pd.to_datetime(combined_df["csv_creation_date"], dayfirst=True)


In [ ]:
# Sütun sıralamasını düzenle
combined_df = combined_df[["Mekan_Adı_Yeni", "Kullanıcı_Adı", "Yorum", "Puan", "Yorum_Tarihi_Yeni"]]

# Sütun adlarını değiştir
combined_df = combined_df.rename(columns={
    "Mekan_Adı_Yeni": "Mekan_Adı",
    "Yorum_Tarihi_Yeni": "Yorum_Tarihi"
})

# "Mekan_Adı" sütunundan (1), (2) gibi parantezli sayıları temizle
combined_df["Mekan_Adı"] = combined_df["Mekan_Adı"].str.replace(r"\(\d+\)", "", regex=True).str.strip("_").str.strip()

combined_df.head(3)

## Mekan Detayları ile Yorumların Birleştirilmesi

In [ ]:
tum_restoranlar = pd.read_csv("tüm_restoranlar.csv", sep=',', on_bad_lines='skip')

def temizle_ve_donustur(isim):
    # Türkçe karakterleri normalize et
    isim = unicodedata.normalize("NFKD", isim).encode("ascii", "ignore").decode("utf-8")
    # Küçük harfe çevir
    isim = isim.lower()
    # Tek tırnak vb. özel karakterleri boş yap
    isim = re.sub(r"[^a-z0-9\s]", "", isim)
    # Boşlukları alt çizgiye çevir
    isim = re.sub(r"\s+", "_", isim.strip())
    return isim

tum_restoranlar["Restoran_Adi_Donusturulmus"] = tum_restoranlar["Restoran_Adi"].apply(temizle_ve_donustur)

In [ ]:
tum_restoranlar = tum_restoranlar[['Restoran_Adi_Donusturulmus', 'Kategori', 'Yorum_Sayisi', 'Puan', 'Adres']]
tum_restoranlar.columns = ['Mekan_Adı', 'Restoran_Kategorisi', 'Restoran_Toplam_Yorum_Sayısı', 'Ortalama_Restoran_Puanı', 'Restoran_Adresi']

In [ ]:
tum_restoranlar.head(3)

,Mekan_Adı,Restoran_Kategorisi,Restoran_Toplam_Yorum_Sayısı,Ortalama_Restoran_Puanı,Restoran_Adresi
0,trilye_restoran,Restoran,2947.0,4.5,"Kazım Özalp, Kuleli Sokağı No: 32, 06680 Çanka..."
1,louise_cafe_brasserie_loft,Restoran,1812.0,4.3,"100. Yıl, Filistin Cd. No:37, 06700 Çankaya/An..."
2,lavare_sokak,Restoran,1452.0,4.6,"16/1, Çankaya, Üsküp Cd., 06690 Çankaya/Ankara"


In [ ]:
# "Mekan_Adı" sütununa göre birleştir
merged_df = combined_df.merge(tum_restoranlar, on="Mekan_Adı", how="inner")
merged_df.head(3)

,Mekan_Adı,Kullanıcı_Adı,Yorum,Puan,Yorum_Tarihi,Restoran_Kategorisi,Restoran_Toplam_Yorum_Sayısı,Ortalama_Restoran_Puanı,Restoran_Adresi
0,01_adanali_ismail_ustanin_yeri,güvenç tünay,Mezeler etler tavuk harika\nHizmet süper\nHarika,5.0,2025-07-13,Restoran,767.0,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"
1,01_adanali_ismail_ustanin_yeri,Ali Can Vurgun,Mezeleri temizliği yemeklerin lezzeti harikadi...,5.0,2025-06-20,Restoran,767.0,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"
2,01_adanali_ismail_ustanin_yeri,Pelin Yildirim,Resmen dolandırıldık. Yani bunu başka türlü aç...,1.0,2025-06-20,Restoran,767.0,4.1,"Sancak, Tiflis Cd. No:41, 06550 Çankaya/Ankara"


In [ ]:
merged_df[merged_df['Ortalama_Restoran_Puanı'].isna()]['Mekan_Adı'].unique()

# 2 restoranın ortalama puanı bulunamadığı için NaN olarak kalmış. Bu iki restoran için ortalama puanları manuel olarak ekleyebiliriz.

array(['burger_king', 'kagithane_gece_donercisi'], dtype=object)

In [ ]:
#"burger_king" restoranına ait ortalama puana ulaşılamadığı için bu restoranı kaldırabiliriz. "kagithane_gece_donercisi" için ortalama puanı 4.9 olarak atayabiliriz.

# "burger_king" restoranına ait ve Ortalama_Restoran_Puanı boş olan satırları sil
merged_df = merged_df[~((merged_df["Mekan_Adı"] == "burger_king") & (merged_df["Ortalama_Restoran_Puanı"].isna()))]

# "kagithane_gece_donercisi" için Ortalama_Restoran_Puanı boş olanlara 4.9 ata
mask = (merged_df["Mekan_Adı"] == "kagithane_gece_donercisi") & (merged_df["Ortalama_Restoran_Puanı"].isna())
merged_df.loc[mask, "Ortalama_Restoran_Puanı"] = 4.9

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171042 entries, 0 to 171241
Data columns (total 9 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Mekan_Adı                     171042 non-null  object        
 1   Kullanıcı_Adı                 171041 non-null  object        
 2   Yorum                         171042 non-null  object        
 3   Puan                          171042 non-null  float64       
 4   Yorum_Tarihi                  171042 non-null  datetime64[ns]
 5   Restoran_Kategorisi           171042 non-null  object        
 6   Restoran_Toplam_Yorum_Sayısı  171042 non-null  float64       
 7   Ortalama_Restoran_Puanı       171042 non-null  float64       
 8   Restoran_Adresi               171042 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 13.0+ MB


Bazı mekanların detayları tüm mekanlar dosyasında bulunmadığı için veride azalma yaşanmıştır. Ancak üzerinden çalışılacak nihai dosya "merged_df" olarak sonuçlandırılmıştır.

Sonuç olarak; ön işlemeye hazır 171.042 adet yorum bulunmaktadır.

In [ ]:
merged_df.to_excel("FinalDataFrame.xlsx", index=False, engine='openpyxl')